In [1]:
import cv2
import os
import tensorflow as tf
import numpy as np
from pygame import mixer
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize mixer for sound
mixer.init()
sound = mixer.Sound(r"C:\Users\theor\Downloads\alarm.wav")

# Load Haarcascade files
face = cv2.CascadeClassifier(r"C:\Users\theor\Downloads\Drowsiness detection\Drowsiness detection\haar cascade files\haarcascade_frontalface_alt.xml")
leye = cv2.CascadeClassifier(r"C:\Users\theor\Downloads\Drowsiness detection (1)\Drowsiness detection\haar cascade files\haarcascade_lefteye_2splits.xml")
reye = cv2.CascadeClassifier(r"C:\Users\theor\Downloads\DL GROUP PROJECT\DL GROUP PROJECT\haarcascade_righteye_2splits.xml")

# Load the CNN model
model = tf.keras.models.load_model(r"C:\Users\theor\Downloads\drowsiness_detection_cnn.h5")

# Start Video Capture
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
score = 0
alarm_on = False  # Track if alarm is playing

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face.detectMultiScale(gray, minNeighbors=5, scaleFactor=1.1, minSize=(25, 25))
    left_eye = leye.detectMultiScale(gray)
    right_eye = reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0, height - 50), (200, height), (0, 0, 0), thickness=cv2.FILLED)

    rpred_class = 1  # Default to Open
    lpred_class = 1  # Default to Open

    # Process right eye
    for (x, y, w, h) in right_eye:
        r_eye = frame[y:y + h, x:x + w]
        r_eye = cv2.cvtColor(r_eye, cv2.COLOR_BGR2RGB)
        r_eye = cv2.resize(r_eye, (64, 64))  # Ensure correct size
        r_eye = r_eye / 255.0
        r_eye = np.expand_dims(r_eye, axis=-1)  # Ensure proper shape (64, 64, 1)
        r_eye = np.expand_dims(r_eye, axis=0)   # Add batch dimension

        rpred = model.predict(r_eye)
        rpred_class = np.argmax(rpred, axis=1)[0]
        break  # Only process first detected eye

    # Process left eye
    for (x, y, w, h) in left_eye:
        l_eye = frame[y:y + h, x:x + w]
        l_eye = cv2.cvtColor(l_eye, cv2.COLOR_BGR2RGB)
        l_eye = cv2.resize(l_eye, (64, 64))  # Ensure correct size
        l_eye = l_eye / 255.0
        l_eye = np.expand_dims(l_eye, axis=-1)  # Ensure proper shape (64, 64, 1)
        l_eye = np.expand_dims(l_eye, axis=0)   # Add batch dimension

        lpred = model.predict(l_eye)
        lpred_class = np.argmax(lpred, axis=1)[0]
        break  # Only process first detected eye

    # If both eyes are closed, increase score
    if rpred_class == 0 and lpred_class == 0:
        score += 1
        cv2.putText(frame, "Closed", (10, height - 20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
    else:  # If any eye is open, slowly decrease score
        score = max(0, score - 1)
        cv2.putText(frame, "Open", (10, height - 20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)

    # Display Score
    cv2.putText(frame, 'Score:' + str(score), (100, height - 20), font, 1, (255, 255, 255), 1, cv2.LINE_AA)

    # Alarm Logic
    if score > 25:
        if not alarm_on:  # Play alarm only if it's not already playing
            sound.play(-1)  # Loop alarm
            alarm_on = True
        cv2.rectangle(frame, (0, 0), (width, height), (0, 0, 255), 2)
    else:
        if alarm_on:  # Stop alarm when eyes open again
            sound.stop()
            alarm_on = False

    # Show Video Feed
    cv2.imshow('Drowsiness Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 687ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━